In [1]:
import simple_id
import torch
import os
import time
import gensim
import sys
%load_ext line_profiler

import warnings
warnings.filterwarnings('ignore')
#from helpers import *

In [2]:
title = "Exploration of computers, tech and R"
abstract = 'We used R and Python to do stuff. Then we did more stuff. Our code can be found on Github'
w2vPath = 'w2v.zip'
modelPath = 'data/BiRNN-2-128-260.pt'


d, a = simple_id.makeVarArray(title, abstract, w2vPath, modelPath)

In [16]:
pred

{'weightP': tensor(0.9183, device='cuda:0'),
 'weightN': tensor(-0.9471, device='cuda:0'),
 'probPos': tensor(0.9492),
 'probNeg': tensor(0.1341),
 'prediction': 1}

Previous work: [http://lstm.seas.harvard.edu/client/index.html](http://lstm.seas.harvard.edu/client/index.html)

Current interfaces:

[Word2Vec visual](http://shiny.reidmcy.com/int/)

[Records inspector](http://shiny.reidmcy.com/ntg/)

In [ ]:
title = "Exploration of computers, tech and R"
abstract = 'We used R and Python to do stuff. Then we did more stuff. Our code can be found on Github'
w2vPath = 'w2v/w2v.bin'
modelFname = 'data/BiRNN-2-128-260.pt'

%lprun -f simple_id.makeVarArray a = simple_id.makeVarArray(title, abstract, w2vPath, modelFname)

In [ ]:
import zipfile
import tempfile
import io


with zipfile.ZipFile('w2v.zip', 'w') as myzip:
    for i, word in enumerate(w2v.wv.vocab):
        with tempfile.NamedTemporaryFile() as tmp:
            np.save(tmp, w2v.wv[word])
            tmp.seek(0)
            try:
                myzip.write(
                    filename = tmp.name,
                    arcname = word,
                )
            except:
                print(word)
                pass

In [ ]:
myzip =  zipfile.ZipFile('w2v.zip', 'r')

In [ ]:
w2v.wv.vocab


In [ ]:
with myzip.open('and') as myfile:
    f = io.BytesIO(myfile.read())
    a = np.load(f)
    print(a)

In [ ]:
w2v.wv['test']

In [ ]:
t = tempfile.NamedTemporaryFile()

In [ ]:
t.seek(0)#(b"SDFDFDFGFDD")


In [ ]:
w2v.wv.syn0.

# Setting up enviroment

In [ ]:
w2v = gensim.models.word2vec.Word2Vec.load('w2v/w2v.bin')
Net = torch.load(modelFname).cuda()

In [ ]:
df_sample = pandas.read_csv('data/Other social sciences.csv', error_bad_lines = False)
df_sample.index = df_sample['wos_id']

# Picking example record

In [ ]:
row = df_sample.loc['WOS:000206783400002']

In [ ]:
row_dict = dict(row)
row_dict['title_tokens'] = Tokens[row_dict['wos_id']][0]
row_dict['abstract_tokens'] = Tokens[row_dict['wos_id']][1]


row_dict['title_vecs'] = simple_id.genVecSeq(row_dict['title_tokens'], w2v)
row_dict['abstract_vecs'] = simple_id.genVecSeq(row_dict['abstract_tokens'], w2v)
row_dict['class'] = 1

In [ ]:
predictionDict = Net.predictRow(row_dict)
predictionDict['wos_id'] = row['wos_id']
predictionDict['title'] = row['title']

In [ ]:
predictionDict

# Activations Visualized

In [ ]:
simple_id.compareRows([row_dict], Net)

In [ ]:
row_dict['title_tokens']

# Effects of varying input

In [ ]:
for i in range(len(row_dict['title_tokens'])):
    newDict = {
    'abstract' : row_dict['abstract'],
    'abstract_tokens' : row_dict['abstract_tokens'],
    #'title' : row_dict['title'],
    'title_tokens' : row_dict['title_tokens'][:i + 1],   
    }
    pred = Net.predictRow(newDict, w2v=w2v)
    print("propP is: {:.2f}% with title: '{}'".format(pred['probPos'] * 100, ' '.join(newDict['title_tokens'])))

In [ ]:
for i in range(len(row_dict['abstract_tokens'])):
    newDict = {
    'abstract' : row_dict['abstract'],
    'abstract_tokens' : row_dict['abstract_tokens'][:i + 1],
    'title' : row_dict['title'],
    'title_tokens' : row_dict['title_tokens'],   
    }
    pred = Net.predictRow(newDict, w2v=w2v)
    print("propP is: {:.2f}% with abstract only {} sentences long".format(pred['probPos'] * 100, i + 1))

In [ ]:
preds1 = []
for i in range(len(row_dict['title_tokens'])):
    predT = []
    for j in range(len(row_dict['abstract_tokens'])):
        newDict = {
            'abstract_tokens' : row_dict['abstract_tokens'][:j+1],
            'title_tokens' : row_dict['title_tokens'][:i + 1],   
            }
        pred = Net.predictRow(newDict, w2v=w2v)
        predT.append(float(pred['probPos']))
    preds1.append(predT)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 7))
seaborn.heatmap(preds1, ax = ax, vmin=0, vmax=1, cmap='RdBu')
ax.set_ylabel('Word title Terminated at')
ax.set_xlabel('Number of Sentences in Abstract')
ax.set_yticklabels(reversed(row_dict['title_tokens']),rotation = 35, fontsize = 20)
row_dict['title_tokens']
ax.set_title("Effects of varying input on DNN output probabilties");

In [ ]:
preds = []
for i in range(len(row_dict['title_tokens'])):
    predT = []
    for j in range(len(row_dict['abstract_tokens'])):
        for k in range(len(row_dict['abstract_tokens'][j])):
            newDict = {
                'abstract_tokens' : row_dict['abstract_tokens'][:j] + [row_dict['abstract_tokens'][j][:k+1]],
                'title_tokens' : row_dict['title_tokens'][:i + 1],   
                }
            pred = Net.predictRow(newDict, w2v=w2v)
            predT.append(float(pred['probPos']))
    print("{:.0f}% done".format(i / len(row_dict['title_tokens']) * 100), end = '\r')
    preds.append(predT)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 7))
seaborn.heatmap(preds, ax = ax, vmin=0, vmax=1, cmap='RdBu')
ax.set_ylabel('Word title Terminated at')
ax.set_xlabel('Number of Words in Abstract')
ax.set_yticklabels(reversed(row_dict['title_tokens']),rotation = 35, fontsize = 20)
ax.set_title("Effects of varying Input on DNN output probabilties");

# Interactive Mode Example

In [ ]:
title = "Exploration of computers, tech and R"
abstract = 'We used R and Python to do stuff. Then we did more stuff. Our code can be found on Github'
title2 = "Effects of input elaboration on vocabulary acquisition through reading by Korean learners of English as a foreign language"

abstract2 = """This article investigates whether (a) lexical elaboration (LE), typographical enhancement (TE), or a combination, and (b) explicit OF implicit,LE affect 297 Korean learner's acquisition of English vocabulary. The learners were asked to read one of six versions of an experimental text that contained 26 target words. The study adopted a 2 X 3 MANOVA (:design with TE and LE as two independent variables and form- and meaning-recognition vocabulary posttests as two dependent variables. The TE had two levels, enhanced and unenhanced, and the LE lipid three levels, explicit, implicit, and unelaborated. The results were (a) LE alone did not aid form recognition of vocabulary, (b) explicit LE alone aided meaning recognition of vocabulary, (c) TE alone did not aid form and meaning recognition of vocabulary, (d) LE and TE combined did not aid form recognition of vocabulary, (e) both explicit and implicit LE aided meaning recognition of vocabulary, (f) explicit and implicit LE (lid riot differ in their effect on form and meaning recognition of vocabulary, and (g) whether a text was further enhanced in addition to either explicit or implicit LE did not seem to affect the acquisition of the previously Unknown words' forms or meanings."""

In [ ]:
title = "Exploration of humans, society and R"
abstract = ' Our code can be found on Github. We used methods and techniques to do stuff. Weber Freud and vocabulary acquisition were explored. Then we did more stuff.'

In [ ]:
exampleDict = {
    'title_tokens' : simple_id.tokenizer(title),
    'abstract_tokens' : simple_id.sentinizer(abstract),   
    }
pred = Net.predictRow(newDict, w2v=w2v)
print(float(pred['probPos']))

In [ ]:
predA = makeVaryingArray(exampleDict, Net, w2v)
fig, ax = plt.subplots(figsize = (15, 7))
seaborn.heatmap(predA, ax = ax, vmin=0, vmax=1, cmap='RdBu', annot=True)
ax.set_ylabel('Title went to ...')
ax.set_xlabel('Abstract went to ...')
ax.set_yticklabels(exampleDict['title_tokens'],rotation = 55, fontsize = 20)
ax.set_xticklabels([' '.join(t)[:25] + '...' for t in exampleDict['abstract_tokens']],rotation = 0, fontsize = 10)
ax.set_title("Effects of varying Input on output probabilty of computational")
plt.savefig('images/occ.pdf', format='pdf');

In [ ]:
exampleDict = {
    'title_tokens' : simple_id.tokenizer(title2),
    'abstract_tokens' : simple_id.sentinizer(abstract2),   
    }
pred = Net.predictRow(exampleDict, w2v=w2v)
print(float(pred['probPos']))

In [ ]:
predA = makeVaryingArray(exampleDict, Net, w2v)
fig, ax = plt.subplots(figsize = (15, 7))
seaborn.heatmap(predA, ax = ax, vmin=0, vmax=1, cmap='RdBu')
ax.set_xlabel('Number of Sentences in Abstract')
ax.set_ylabel('Word title Terminated at')
ax.set_yticklabels(reversed(exampleDict['title_tokens']),rotation = 35, fontsize = 20)
ax.set_title("Effects of varying Input on DNN output probabilties");

In [ ]:
exampleDict = {
    'title_tokens' : simple_id.tokenizer(title2),
    'abstract_tokens' : simple_id.sentinizer(abstract2 + ' Computer github code model python R C software simulation'),   
    }
pred = Net.predictRow(exampleDict, w2v=w2v)
print(float(pred['probPos']))

In [ ]:
predA = makeVaryingArray(exampleDict, Net, w2v)
fig, ax = plt.subplots(figsize = (15, 7))
seaborn.heatmap(predA, ax = ax, vmin=0, vmax=1, cmap='RdBu')
ax.set_xlabel('Number of Sentences in Abstract')
ax.set_ylabel('Word title Terminated at')
ax.set_yticklabels(reversed(exampleDict['title_tokens']),rotation = 35, fontsize = 20)
ax.set_title("Effects of varying Input on DNN output probabilties");

# Making the images for a bunch of examples

In [ ]:
for i, row in df_sample.iloc[:5].iterrows():
    print(row['title'])
    row_dict = dict(row)
    row_dict['title_tokens'] = Tokens[row_dict['wos_id']][0]
    row_dict['abstract_tokens'] = Tokens[row_dict['wos_id']][1]
    predA = makeVaryingArray(row_dict, Net, w2v)
    fig, ax = plt.subplots(figsize = (15, 7))
    seaborn.heatmap(predA, ax = ax, vmin=0, vmax=1, cmap='RdBu')
    ax.set_xlabel('Number of Sentences in Abstract')
    ax.set_ylabel('Word title Terminated at')
    ax.set_yticklabels(reversed(row_dict['title_tokens']),rotation = 35, fontsize = 20)
    ax.set_title("Effects of varying Input on DNN output probabilties:\n{}".format(row['title']))
    fig.tight_layout()
    plt.savefig('images/pred_var_{}.pdf'.format(row['title'][:50].replace(' ', '_')), format = 'pdf')
    
for i, row in df_sample.iloc[-5:].iterrows():
    print(row['title'])
    predA = makeVaryingArray(row_dict, Net, w2v)
    fig, ax = plt.subplots(figsize = (15, 7))
    seaborn.heatmap(predA, ax = ax, vmin=0, vmax=1, cmap='RdBu')
    ax.set_ylabel('Word title Terminated at')
    ax.set_yticklabels(reversed(row_dict['title_tokens']),rotation = 35, fontsize = 20)
    ax.set_xlabel('Number of Sentences in Abstract')
    ax.set_title("Effects of varying Input on DNN output probabilties:\n{}".format(row['title']))
    fig.tight_layout()
    plt.savefig('images/pred_var_{}.pdf'.format(row['title'][:50].replace(' ', '_').replace('/', '')), format = 'pdf')

# Next steps

+ Get interface with R setup
+ Make interactive
+ Add real explanations
+ Maybe add more modes

Is it truthful?

Is it functional?

Is it beautiful?

Is it insightful?

Is it enlightening?